In [1]:
import requests     
import json         
import pandas as pd 
import time
from fuzzywuzzy import fuzz

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
path = "./Data/withFIGI.csv"

df = pd.read_csv(path)  
df

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3,5,6,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,country-Inv,...,longName-FIGI,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI
0,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,united states,...,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
1,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,united kingdom,...,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
2,7063,Japan,JPY,TSE,JP3160590000,A Dot Co Ltd,Common Stock,2064.000,2022-08-03,japan,...,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
3,2018,Hong Kong,HKD,HK,KYG2953R1149,AAC Technologies Holdings Inc,Common Stock,14.580,2022-08-03,hong kong,...,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
4,2686,Hong Kong,HKD,HK,KYG000371040,AAG Energy Holdings Ltd,Common Stock,1.410,2022-08-03,hong kong,...,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17506,NaN,NaN,NaN,NaN,US98979J1097,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17507,NaN,NaN,NaN,NaN,CA98977K1084,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17508,NaN,NaN,NaN,NaN,CA98977R1038,NaN,NaN,NaN,NaN,NaN,...,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock
17509,NaN,NaN,NaN,NaN,GB00BMHTHT14,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## EOD vs 19.7k

In [3]:

df_Name = df[['ISIN', 'Name_197', 'Name-EOD']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'Name-EOD']].dropna(subset=['Name-EOD'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['Name-EOD'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]

13016
progress: 0.00%
progress: 10.00%
progress: 19.99%
progress: 29.99%
progress: 39.98%
progress: 49.98%
progress: 59.97%
progress: 69.97%
progress: 79.96%
progress: 89.96%
progress: 99.95%


,ISIN,Name_197,Name-EOD,fuzz_socre1
38,JP3108120001,ACRODEA INC,Why How Do Co Inc,50
53,LU1250154413,ADO PROPERTIES SA,Adler Group SA,58
78,US0077371096,AEROCENTURY CORP,Mega Matrix Corp,25
165,US0185811082,ALLIANCE DATA SYSTEMS CORP,Bread Financial Holdings Inc,26
223,US0259321042,AMERICAN FINANCIAL GROUP INC,Aquatic Foods Group plc,51
...,...,...,...,...
12934,US98585L1008,YIREN DIGITAL LTD - SPS ADR,Yirendai Ltd,49
12942,CNE100002W92,YOUON TECHNOLOGY CO LTD-A,Changzhou Youon Pubilic Bicycle System Co Ltd,49
12970,CH0276534614,ZEHNDER GROUP AG-RG,Zehnder,54
12996,CNE000000FL2,ZHONGTIAN FINANCIAL GROUP -A,Zhongtian Urban Development Group Co Ltd,54


## Investing.com vs 19.7k

In [4]:
df_Name = df[['ISIN', 'Name_197', 'full_name-Inv']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'full_name-Inv']].dropna(subset=['full_name-Inv'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['full_name-Inv'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]


6141
progress: 0.00%
progress: 10.00%
progress: 20.00%
progress: 30.00%
progress: 39.99%
progress: 49.99%
progress: 59.99%
progress: 69.99%
progress: 79.99%
progress: 219.96%


,ISIN,Name_197,full_name-Inv,fuzz_socre1
246,IL0010806136,ANALYST I.M.S.,Analyst IMS Investment Management Services Ltd,44
562,RU000A0HL5M1,BELUGA,Beluga Group PAO,55
2779,US5006432000,KORN FERRY,Korn/Ferry International,59
2965,IL0004730177,LUZON GROUP,Amos Luzon Development and Energy Group Ltd,41
3794,PLPKO0000016,PKO BANK POLSKI SA,Powszechna Kasa Oszczednosci Bank Polski SA,56
4167,FR0006239109,SCBSM,Soc Centrale Bois Scieries Manche SA,15


## IB vs 19.7k

In [5]:
df_Name = df[['ISIN', 'Name_197', 'longName-IB']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'longName-IB']].dropna(subset=['longName-IB'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['longName-IB'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]

16912
progress: 0.00%
progress: 10.00%
progress: 20.00%
progress: 30.00%
progress: 40.00%
progress: 49.99%
progress: 59.99%
progress: 69.99%
progress: 89.99%
progress: 99.99%


,ISIN,Name_197,longName-IB,fuzz_socre1


## FIGI vs 19.7k

In [9]:
df_Name = df[['ISIN', 'Name_197', 'longName-FIGI']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'longName-FIGI']].dropna(subset=['longName-FIGI'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['longName-FIGI'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]

15485
progress: 0.00%
progress: 10.00%
progress: 19.99%
progress: 29.99%
progress: 39.99%
progress: 49.98%
progress: 59.98%
progress: 69.98%
progress: 79.97%
progress: 99.97%


,ISIN,Name_197,longName-FIGI,fuzz_socre1
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
38,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,32
53,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,58
78,US0077371096,AEROCENTURY CORP,MEGA MATRIX CORP,25
121,JP3102000001,AISIN SEIKI CO LTD,AISIN CORP,57
...,...,...,...,...
16577,US74838C1062,QUIKSILVER INC,BOARDRIDERS INC,21
16581,US7472623013,QVC INC 6 3/8 09/13/67 PFD,QVC INC,42
16690,SE0010520155,ROLLING OPTICS HOLDING AB,ICTA AB,31
17055,US7847882005,"SYNTHETIC FIXED-INCOME SEC STRATS 2004-2, 6.50...",STRATS-FON-04-2,36
